In [1]:
pip install tf-keras pandas torch transformers scikit-learn ipywidgets sentence_transformers tqdm


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm

In [3]:
# Load the dataset
df = pd.read_csv('ISOT10000.csv')  # Replace with your CSV file path
df = df.dropna(subset=['label'])
# Extract relevant columns
text_data = df['text'].fillna('').tolist()
title_data = df['title'].fillna('').tolist()
combined_data = [f"{title} {text}" for title, text in zip(title_data, text_data)]
labels = df['label'].tolist()

# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
print(df['label'].value_counts())

label
0    5000
1    4999
Name: count, dtype: int64


In [4]:
# Define function to get BERT embeddings
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # Pool to get a single embedding



In [5]:

#model = SentenceTransformer('all-MiniLM-L6-v2')


# Get embeddings for each text
#embeddings = [get_bert_embedding(title) for title in title_data]

embeddings = [get_bert_embedding(text) for text in tqdm(combined_data)]


#embeddings = [model.encode(text) for text in text_data]



  0%|          | 0/9999 [00:00<?, ?it/s]

In [6]:
# Split data into training and test sets
X_temp, X_test, y_temp, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 20% of total for validation

# Check for NaN values in y_train and y_test after splitting
if any(pd.isna(y_train)) or any(pd.isna(y_test)):
    print("Warning: NaN values detected in target variable after train-test split.")



# Scale embeddings
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)
# Increase max_iter in LogisticRegression / Train a classifier
#clf = RandomForestClassifier(n_estimators=100, random_state=42)
#clf = LogisticRegression(max_iter=500)  # Increase iterations

ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True))
], voting='soft')
ensemble.fit(X_train_scaled, y_train)

# Validate the model
y_val_pred = ensemble.predict(X_val_scaled)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# Test the model
y_test_pred = ensemble.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Print individual classifier votes for the test set
print("Classifier Votes for Test Samples:")
for i, (lr_vote, rf_vote, svc_vote) in enumerate(zip(
    ensemble.named_estimators_['lr'].predict(X_test_scaled),
    ensemble.named_estimators_['rf'].predict(X_test_scaled),
    ensemble.named_estimators_['svc'].predict(X_test_scaled)
)):
    print(f"Sample {i+1}: LogisticRegression={lr_vote}, RandomForest={rf_vote}, SVC={svc_vote}, Final Vote={y_test_pred[i]}")

# Print detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))


Validation Accuracy: 99.45%
Test Accuracy: 99.35%
Classifier Votes for Test Samples:
Sample 1: LogisticRegression=1, RandomForest=1, SVC=1, Final Vote=1
Sample 2: LogisticRegression=0, RandomForest=0, SVC=0, Final Vote=0
Sample 3: LogisticRegression=0, RandomForest=0, SVC=0, Final Vote=0
Sample 4: LogisticRegression=0, RandomForest=0, SVC=0, Final Vote=0
Sample 5: LogisticRegression=1, RandomForest=1, SVC=1, Final Vote=1
Sample 6: LogisticRegression=0, RandomForest=0, SVC=0, Final Vote=0
Sample 7: LogisticRegression=1, RandomForest=1, SVC=1, Final Vote=1
Sample 8: LogisticRegression=0, RandomForest=0, SVC=0, Final Vote=0
Sample 9: LogisticRegression=1, RandomForest=1, SVC=1, Final Vote=1
Sample 10: LogisticRegression=1, RandomForest=1, SVC=1, Final Vote=1
Sample 11: LogisticRegression=1, RandomForest=1, SVC=1, Final Vote=1
Sample 12: LogisticRegression=1, RandomForest=1, SVC=1, Final Vote=1
Sample 13: LogisticRegression=1, RandomForest=1, SVC=1, Final Vote=1
Sample 14: LogisticRegressi

In [7]:
# Drop any unnamed columns (columns with no header) if they exist
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Add predictions to original dataframe
df['prediction'] = ensemble.predict(embeddings)
#df['prediction'] = df['prediction'].apply(lambda x: 'real' if x == 1 else 'fake')

# Save results
df.to_csv('bert_isotcombined.csv', index=False)
